# Inspecting synthetic galaxy catalogs

The purpose of this notebook is to run create a large sample of synthetic cluster members.  Later on these synthetic cluster members will be used to populate synthetic galaxy clusters


## Objectives




## Setup

This notebook relies on two prerequisites:

    * Data files from the previous calculation step. That is the output of the rejection sampling
    * synthetic software package. The host package of this tutorial


## Output

The outuput of this claculation is a very large set of synthetic galaxies sampled from the
    
    * galaxy clusters member model
    * reference line of sight model
    
## Contact

In case of questions, contact me at t.varga@physik.lmu.de

In [1]:
from importlib import reload
import fitsio as fio
import numpy as np
import pandas as pd

import healpy as hp
import copy
import sys
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline

import sklearn.decomposition as decomp


import matplotlib as mpl
import subprocess as sp
import scipy.interpolate as interpolate
import pickle as pickle

import multiprocessing as mp

import synthetic.tools as tools
import synthetic.emulator.emulator as emulator
import synthetic.emulator.indexer as indexer
import synthetic.emulator.reader as reader

# Preparation

## Setting up the file structure

The data files for this example calculation are pre packaged, and should be downloaded from a link provided upon request.

    1 dc2-alpha_concentric_sample-v01_test-03.tar.gz
    2 dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5
    3 dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5
    
These should be downloaded and placed in a file structure such that

    /root/
    |----/resamples/ 
    |----/dc2-alpha_concentric_sample-v01_test-03.tar.gz
    |----/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5
    |----/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5
    
from within the root folder, extract the .tar.gz file using the command

    tar xzf dc2-alpha_concentric_sample-v01_test-03.tar.gz -C  resamples --strip-components 1    
    
This should yield a file structure as below
 
     /root/
    |----/resamples/ 
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run0_1846435878_rbin0.p
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run0_1846435878_rbin0_samples.fits
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run0_1846435878_rbin0_scores.fits
            .
            .
            .
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run3_664487101_rbin3.p
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run3_664487101_rbin3_samples.fits
    |--------------/dc2-alpha_concentric_sample-v01_test-03_run3_664487101_rbin3_scores.fits            
    |----/dc2-alpha_concentric_sample-v01_test-03.tar.gz
    |----/dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5
    |----/dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5

In [4]:
# YOU SHOULD CHANGE THE ROOT PATH TO YOUR OWN PATH
root_path = "/e/ocean1/users/vargatn/LSST/DC2_1.1.4/clusters_v01/"
deep_data_path = root_path + "dc2_cluster_sim_cutouts/cosmoDC2_v1.1.4_refpixels.h5"
wide_data_path = root_path + "dc2_cluster_sim_cutouts/clust_dc2-sim-LOS_v1.h5"

## Configuration 

There are two primary inputs for this calculation: A low dimensional anchor distribution, and a high dimensional augmentation distribution. The distributions are estimated from datasets via KDE, and the dimensionality refers to the number of columns (features) in each dataset.

Both datasets are extracted from a catalog through a series of pre-processing steps. These steps are:

    (1) a shorthand name for each feature e.g. MAG_I, which can replace a more complicated name often encountered in astronomical databases e.g. mag_auto_deblend_mcal_v2_iband. This gives the user the task to identify columns in different catalogs as conceptually the same quantity, such as calibrated i-band magnitudes in different observations of the same *kind* of objects should be the same quantity, even if the catalog columns have different names
    
    (2) an instruction for transforming the catalog column. E.g. colors are defined as g - r color is specified as ("mag_g", "mag_r", "-"). A constant transformation is also allowed, e.g. z+1 is specified as ("z", 1, "+")
    
    (3) logarithmic scaling for each column, which is applied after the previous transformation. The boolean list for this should be specified in the same order as the column names in the first step
    
    (4) limits for clipping each column to a range. The list of tuples of limits for this should be specified in the same order as the column names in the first step.


Since the current example is defined for a simulated sky survey, both distributions are derived from the same galaxy catalog, so their selection in this notebook are identical. **NOTE This is only true for this notebook, in order to compare predictions to the ground truth**

In [16]:
deep_smc_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass")        
    ],
    "logs": [False, True, False, False, False, False, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16)],
}

In [15]:
wide_cr_settings = {
    "columns": [
        ("GABS", ("ellipticity_1_true", "ellipticity_2_true", "SQSUM")),
        ("SIZE", "size_true"),
        ("MAG_I", "mag_i"),
        ("COLOR_G_R", ("mag_g", "mag_r", "-")),
        ("COLOR_R_I", ("mag_r", "mag_i", "-")),
        ("COLOR_I_Z", ("mag_i", "mag_z", "-")),
        ("STELLAR_MASS", "stellar_mass"),
        ("HALO_MASS", "halo_mass"),
        ("LOGR", "R"),        
    ],
    "logs": [False, True, False, False, False, False, True, True, True],
    "limits": [(0., 1.), (-1, 5), (17, 25), (-1, 3), (-1, 3), (-1, 3), (10**3, 10**13), (10**9, 10**16), (1e-3, 16)],
}


Only a few columns are actually used to connect the anchor and augmentation distributions

# Loading data from disk

## Loading the regular data tables

First load the two primary obsered datasets, since this is a simulated scenarion both are comprehensive and contain all relevant quantities 

In [12]:
refpixel = pd.read_hdf(deep_data_path, key="data")
table = pd.read_hdf(wide_data_path, key="data")

Then extract the desired columns and features

In [13]:
# loading random point reference data, in this case this is identical to the agumentation data
tmp_wide_cr_settings = wide_cr_settings.copy()
_wide_cr_settings_rands = emulator.construct_deep_container(refpixel, tmp_wide_cr_settings)

# loading augmentation catalogs, in this case this is a randomly chosen helpix pixel in the simulation
_deep_smc_settings = emulator.construct_deep_container(refpixel, deep_smc_settings)

# loading anchor data, in this case this is the cluster in the simulation
tmp_wide_cr_settings = wide_cr_settings.copy()
_wide_cr_settings_clust = emulator.construct_deep_container(table, tmp_wide_cr_settings)


(1513572, 9)
(1513572,)
(1513572, 8)
(1513572,)
(1009658, 9)
(1009658,)


## Loading the results of rejection sampling

Reading from file the results of the rejection sampling

These consist of the random samples which were evaluated, and of the P(x) values, that is the scores corresponding to those sampled data points.

Due numerical optimization, these files are saved separately. The different files are read separately, and then concatenated into a large data frame for each nested radial shell.

In [ ]:
samples = []
scores = []
for rbin in np.arange(4):
    print(rbin)
    expr = root_path + "resamples/dc2-alpha_concentric_sample-v01_test-03/dc2-alpha_concentric_sample-v01_test-03_run0*_rbin" + str(rbin) + "*samples.fits" 

    fnames_samples = np.sort(glob.glob(expr))
    fnames_scores = []
    for fname in fnames_samples:
        fnames_scores.append(fname.replace("samples.fits", "scores.fits"))

    samples_sep = []
    scores_sep = []
    for i, fname in enumerate(fnames_samples):
#         print(fname)
        samples_sep.append(fio.read(fname))
        scores_sep.append(fio.read(fnames_scores[i]))
        
    samples.append(np.hstack(samples_sep))
    scores.append(np.hstack(scores_sep))

In [ ]:
mag_lims = (17, 22.5)
r_lims_all = [(-1.5, -0.5), (-0.5, 0.), (0, 0.5), (0.5, 1.0)]
redges = [-1.5, -0.5, 0., 0.5, 1.0]
rareas = np.array([np.pi*((10**redges[i+1])**2. - (10**redges[i])**2.) for i in np.arange(len(redges)-1)])

In [ ]:
magcol = "mag_i"
ii = ((table[magcol] > mag_lims[0]) & (table[magcol] < mag_lims[1]))

clust_los_nums = np.histogram(np.log10(table[ii]["R"]), bins=redges)[0] / 41 # / nc

ii = ((refpixel[magcol] > mag_lims[0]) & (refpixel[magcol] < mag_lims[1]))
surfdens = len(refpixel[ii]) / hp.nside2pixarea(32, degrees=True) / 3600 / 3
rands_los_nums = surfdens * rareas
# rands_los_nums = np.histogram(np.log10(refpixel[ii]["R"]), bins=redges)[0] / rareas#* ratio# / nr
nratios = clust_los_nums / rands_los_nums

In [ ]:
ifields2 = []
iclusts2 = []
i2ds2 = []
for rbin in np.arange(4):
    print(rbin)
    _ifield, _iclust, _i2d = reader.result_reader2(samples[rbin], scores[rbin], nratio=nratios[rbin], m_factor=100, seed=rbin)
    ifields2.append(_ifield)
    iclusts2.append(_iclust)
    i2ds2.append(_i2d)

In [ ]:
csamples = []
for rbin in np.arange(4):
    print(rbin)
    tab = pd.DataFrame.from_records(samples[rbin][iclusts2[rbin]].byteswap().newbyteorder())
    tab.drop('index', axis=1, inplace=True)
    kde = emulator.KDEContainer(tab)
    kde.standardize_data()
    kde.construct_kde(0.1)
    _csample = kde.random_draw(4e6)
    csamples.append(_csample)

In [ ]:
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01.h5"

In [ ]:
tools.save(opath, csamples)

In [ ]:
ctab = _wide_cr_settings_clust["container"].data
rtab = _wide_cr_settings_rands["container"].data

In [ ]:
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01_ctab.h5"
ctab.to_hdf(opath, key="data")
opath = "/e/ocean1/users/vargatn/LSST/SYNTHETIC/csamples_nov_dev_01_rtab.h5"
rtab.to_hdf(opath, key="data")